In [1]:
# General imports
import numpy as np
import matplotlib.pyplot as plt

def colors(N_COLORS,colorMap='summer'):
    cmap = plt.cm.get_cmap(colorMap)
    return [cmap(i) for i in np.linspace(0,1,N_COLORS)]

# __ Numerical solution of integral equations__

Methods to obtain a better guess for a trial solution $\mathbf{x}$ of a nonlinear function $F: \mathbb{R}^n \rightarrow \mathbb{R}^n$


 * ## Fixed–point iteration

Convert the transcendental equation $F(\mathbf{x}) = 0 \rightarrow G[\mathbf{x}] = \mathbf{x}$.

_Main idea_: Solve iteratively as $\mathbf{x}_{i+1} = G(\mathbf{x}_{i})$

Example:
$f(x) = 0 \Leftrightarrow x^2 - \sqrt[]{x + 10} = 0$ can be written as $g(x) = x \Leftrightarrow \sqrt[]{x + 10}\,/\,x = x$

In [2]:
def g(x):
    return np.sqrt(x+10)/x

def f(x):
    return x - np.sqrt(x+10)/x

solution = 1.85558
guess = 0.5

In [3]:
# fixed-point plotting
grid = np.linspace(1e-10,g(guess)+0.1,100)

from matplotlib import animation, rc
from IPython.display import HTML

fig, (ax1,ax2) = plt.subplots(1,2)
line, = ax1.plot([], [], lw=2)

ax1.set_xlim((grid[0], grid[-1]))
ax1.set_ylim((0, g(guess)+0.1))

ax1.plot(grid,g(grid),color='royalblue')
ax1.text(1.1, 4.6, r'$g(x)$', fontsize=15, color='royalblue')
ax1.plot(grid,grid,color='tomato')
ax1.text(0.8, 0.5, r'$x$', fontsize=15, color='tomato')

def init():
    line.set_data([], [])
    return (line,)


# Animation function
frames=50
color_list = colors(frames)

fixed_point_guess = guess
res = []

def animate(i):
    global solution
    global fixed_point_guess
    global res
    
    # Get new function
    new_guess = g(fixed_point_guess)
    
    # Type of plot
    if i%2 is 0:
        ax1.plot((fixed_point_guess,fixed_point_guess), (new_guess,fixed_point_guess), color=color_list[i])
        
        # Calculate residual
        res_ = abs(solution-new_guess)
        res += [res_]
        ax2.plot(range(len(res)),res, color='tomato')
        
    else:
        ax1.plot((fixed_point_guess,new_guess), (new_guess,new_guess), color=color_list[i-1])
        fixed_point_guess = new_guess
            
    return (line,)


ax2.set_xlim((0,frames/2))
# ax2.set_ylim(0,10)
ax2.set_yscale('log')
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=150, blit=True)
HTML(anim.to_html5_video())

## __ Idea: Relax the solution __ ##

Solve iteratively $\mathbf{x}_{i+1} = \alpha\, G(\mathbf{x}_{i}) + (1-\alpha)\,\mathbf{x}_{i}$ for $\alpha \in \,]0,1)$

In [22]:
alpha = 0.5

In [23]:
# relaxed plotting

def g_relaxed(x):
    return alpha * g(x) + (1-alpha) * x

# Plotting base
grid = np.linspace(1e-10,g(guess)+0.1,100)

from matplotlib import animation, rc
from IPython.display import HTML

fig, (ax1,ax2) = plt.subplots(1,2)
line, = ax1.plot([], [], lw=2)

ax1.set_xlim((grid[0], grid[-1]))
ax1.set_ylim((0, g(guess)+0.1))

ax1.plot(grid,g(grid),color='royalblue')
ax1.text(1.1, 4.6, r'$g(x)$', fontsize=15, color='royalblue')
ax1.plot(grid,grid,color='tomato')
ax1.text(0.8, 0.5, r'$x$', fontsize=15, color='tomato')


def init():
    line.set_data([], [])
    return (line,)

# Animation function
frames=20
color_list = colors(frames)

res = []
relaxed_fixed_point_guess = guess

def animate(i):
    global solution
    global relaxed_fixed_point_guess
    global res
    
    # Get new function
    new_guess = g_relaxed(relaxed_fixed_point_guess)
    
    # Type of plot
    if i%2 is 0:
        new_guess = g(relaxed_fixed_point_guess)
        ax1.plot((relaxed_fixed_point_guess,relaxed_fixed_point_guess), (new_guess,relaxed_fixed_point_guess)\
                 , color=color_list[i])
        
        # Calculate residual
        res_ = abs(solution-new_guess)
        res += [res_]
        ax2.plot(range(len(res)),res, color='tomato')
        
    else:
        ax1.plot((relaxed_fixed_point_guess,new_guess), (g(relaxed_fixed_point_guess),new_guess), color=color_list[i-1])
        relaxed_fixed_point_guess = new_guess
            
    return (line,)

ax2.set_xlim((0,frames/2))
# ax2.set_ylim(0,10)
ax2.set_yscale('log')
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=150, blit=True)
HTML(anim.to_html5_video())

 * ## Newton's method iteration

Consider a nonlinear function $F: \mathbb{R}^n \rightarrow \mathbb{R}^n$


_Main idea_: Find iteratively the root of the tangent plane of $F$:

$0 = F(\mathbf{x}_i) \cong F(\mathbf{x}_i) + F^{\prime}(\mathbf{x}_i) (\mathbf{x}_{i+1} - \mathbf{x}_i)$



Example:
$f(x) = 0 \Leftrightarrow x^2 - \sqrt[]{x + 10} = 0$.


In [9]:
# Newton's method
from scipy.optimize import fsolve

from scipy.optimize import fsolve

def div_f(x):
    eps = 1e-6
    return (f(x+eps)-f(x))/eps

def tan_f(x,x0):
    fx0 = f(x0)
    return fx0 + div_f(x0) * (x - x0)

def func(x,a):
    return f(a) + div_f(a)*(x-a)

def new_x(a):
    return fsolve(func,0.1,(a))[0]

# helper for plotting gradient
def tan_f_is_zero(x0):
    return fsolve(tan_f,-0.1,x0)[0]

In [10]:
# Plotting base
upB = 3
lwB = 0.4
grid = np.linspace(lwB,upB,100)

from matplotlib import animation, rc
from IPython.display import HTML

fig, (ax1,ax2) = plt.subplots(1,2)
line, = ax1.plot([], [], lw=2)

#ax1.set_xlim((grid[0], grid[-1]))
#ax1.set_ylim((-5, 5))

ax1.plot(grid,f(grid),color='royalblue')
ax1.text(0.8, -4, r'$f(x)$', fontsize=15, color='royalblue')
ax1.axhline(0, color='tomato')
ax1.text(2.5, -0.7, r'$0$', fontsize=15, color='tomato')

def init():
    line.set_data([], [])
    return (line,)

# Animation
color_list = colors(frames)

res = []
frames=20
newton_guess = guess

tan_eps = 0.2

def animate(i):
    global upB
    global lwB
    global solution
    global newton_guess
    global res
    
    if i%2==0:
        # calculate f(guess)
        f_guess = f(newton_guess)
        
        # calculate missing function
        if newton_guess > upB:
            new_grid = np.linspace(upB,newton_guess,100)
            upB = newton_guess
            ax1.plot(new_grid, f(new_grid), color='royalblue')
        if newton_guess < lwB:
            new_grid = np.linspace(newton_guess,lwB,100)
            lwB = newton_guess
            ax1.plot(new_grid, f(new_grid), color='royalblue')

        ax1.plot((newton_guess,newton_guess), (0, f_guess), color='darkmagenta')
    else:
        # plot tangential
        zero_tan = tan_f_is_zero(newton_guess)
        if zero_tan > newton_guess:
            tan_grid = [newton_guess - tan_eps,zero_tan + tan_eps]
        else:
            tan_grid = [zero_tan - tan_eps ,newton_guess + tan_eps]
        ax1.plot(tan_grid,list(map(lambda x: tan_f(x,newton_guess),tan_grid)), color='green')
        
        # new solution
        newton_guess = new_x(newton_guess)
        
        # calculate residual
        res_ = abs(solution-newton_guess)
        res += [res_]
        ax2.plot(range(len(res)),res, color='tomato')
        
    return (line,)

ax2.set_xlim((0,frames/2))
ax2.set_yscale('log')
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=600, blit=True)
HTML(anim.to_html5_video())

## __Fundamental Problem__

For a ___high dimensional___ transcendental function $F$, we _cannot_ afford to calculate the jacobian $\mathbf{J}$

$J_{i j} = \frac{\partial}{\partial x_j} F_i$.

In practice, a new solution via Newton's method is

$\mathbf{u}^{i+1} = \mathbf{u}^i - \mathbf{J}^{-1}(\mathbf{u}^i) \mathbf{F}(\mathbf{u}^i)$
which can be obtained in 2 steps:

* Solve $\mathbf{J}(\mathbf{u}^i) \delta \mathbf{u}^i$ = $-\mathbf{F}(\mathbf{u}^i)$ ( $A x = b$ problem to solve)
* Update $\mathbf{u}^{i+1} = \mathbf{u}^i + \delta \mathbf{u}^i$

## __ Krylov subspace methods __##

allow the computation of $\delta\mathbf{u}$ without ___ever___ assembling the Jacobian!

Note that the lhs of the linear system can be written as

$\mathbf{J}(\mathbf{u}^i) \delta \mathbf{u}^i =
\frac{1}{\epsilon}\left[\,\mathbf{F}(\mathbf{u}^i + \epsilon\, \delta \mathbf{u}^i) - \mathbf{F}(\mathbf{u}^i)\,\right]$ (taylor)

Each trial for $\delta \mathbf{u}^i$ only requires 2 function evaluations!